In [198]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Túto úlohu som robil v Google Colab, knižnica na trénovanie modelu strojového učenia váži pomerne veľa a nechcel som ju sťahovať do svojho lokálneho počítača.

In [199]:
file_path = '/content/drive/MyDrive/dentsu JDA - zadanie 2kolo.xlsx'

In [200]:
import pandas as pd

sheets_dict = pd.read_excel(file_path, sheet_name=None)

In [201]:
df_sheet4 = sheets_dict["bonus zadanie - forecast SOM"]
df_sheet4.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,GRP 30s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2020,2021,2022,2023,01-09/2024,f2024,f2025,f2026,f2027
3,KOFT / BROWN-FORMAN,203.175,179.85,248.25,865.311595,892.03418,NaN,NaN,NaN,NaN
4,GAS FAMILIA,738.77486,566.4675,838.91438,682.78038,90.0476,NaN,NaN,NaN,NaN
5,ITALMARKET,NaN,250.65,958.83,736.31,416.34,NaN,NaN,NaN,NaN
6,MASPEX,NaN,NaN,NaN,989.55,335.625,NaN,NaN,NaN,NaN
7,MAST-JAEGERMEISTER,4389.99992,3692.38311,4885.17833,4616.6061,3434.94896,NaN,NaN,NaN,NaN
8,PERNOD RICARD,3422.74524,4298.01866,5039.73723,4672.76924,2998.27975,NaN,NaN,NaN,NaN
9,ST.NICOLAUS,1226.26715,2334.23998,2672.58618,2167.69726,1286.66357,NaN,NaN,NaN,NaN


In [202]:
data = df_sheet4.iloc[15:, :5]
data.columns = ['Advertiser', 'Brand', 'Year', 'Month', 'GRP_30s']
data.dropna(subset=['Advertiser', 'GRP_30s'], inplace=True)

In [203]:
data.head(3)

,Advertiser,Brand,Year,Month,GRP_30s
15,GAS FAMILIA S.R.O.,VODKA GORAL,2020,január,1.05
16,GAS FAMILIA S.R.O.,FAMILIA PREMIUM,2020,január,1.05
17,GAS FAMILIA S.R.O.,ORIGINAL SPIS,2020,január,1.05


Mojím riešením je vytvoriť model strojového učenia, ktorý predpovedá budúce hodnoty. (Dlho som dostával neadekvátne hodnoty, až kým som si neuvedomil, že problém spočíva v tom, že v niektorých mesiacoch v predloženom dátovom rámci neboli podané žiadne inzeráty. Neskôr som všetky chýbajúce mesiace ručne doplnil hodnotou 0 a model fungoval dobre).

### Najdôležitejšou úlohou bolo vybrať správny model. 

Vybral som si **Prophet**

**Prophet** je knižnica na predpovedanie časových radov vyvinutá spoločnosťou Facebook. Je navrhnutá na spracovanie a analýzu časových údajov a umožňuje vytvárať predpovede na základe historických údajov. 

Výhody tohto riešenia oproti iným automatizovaným možnostiam, ktoré som mohol získať napríklad prostredníctvom Exellu:

- Jednoduchosť používania: Prophet je navrhnutý s ohľadom na použiteľnosť pre používateľov, ktorí nemajú hlboké znalosti štatistiky. Knižnica umožňuje rýchlo získať adekvátne prognózy s minimálnym úsilím.

- Flexibilita: Prophet dokáže zohľadniť sezónnosť (napr. ročné cykly), sviatky a iné faktory ovplyvňujúce údaje.

- Spracovanie odľahlých hodnôt: Prophet dokáže spracovať chýbajúce hodnoty v údajoch a automaticky spracovať odľahlé hodnoty, vďaka čomu je odolný voči nepresnostiam v údajoch.

- Vizualizácia: Knižnica ponúka vstavané funkcie na vizualizáciu nespracovaných údajov aj prognóz, čo uľahčuje analýzu výsledkov.

In [204]:



months_sk = {
    'január': '01', 'február': '02', 'marec': '03', 'apríl': '04', 'máj': '05', 'jún': '06',
    'júl': '07', 'august': '08', 'september': '09', 'október': '10', 'november': '11', 'december': '12'
}

data['Month'] = data['Month'].map(months_sk)




In [205]:
data.Advertiser.unique()

array(['GAS FAMILIA S.R.O.', 'PERNOD RICARD SLOVAKIA S.R.O.',
       'MAST-JAEGERMEISTER SK S.R.O.',
       'REMY COINTREAU SLOVAKIA S.R.O._OLD', 'CORNER SK S.R.O.',
       'ST.NICOLAUS A.S.', 'STOCK SLOVENSKO S.R.O.', 'KOFT S.R.O.',
       'ITALMARKET SLOVAKIA A.S.', 'RUDOLF JELINEK SLOVAKIA S.R.O.',
       'VITIS PEZINOK', 'WINESHOP.SK S.R.O.',
       'FRUIT DISTILLERY COOPERATION', '1 DAY S.R.O.',
       'MASPEX SLOVAKIA S.R.O.', 'PAMPERO ANIVERSARIO',
       'BROWN-FORMAN SLOVAKIA S.R.O.', 'BUMBU RUM COMPANY',
       'KARLOFF S.R.O.'], dtype=object)

In [214]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from prophet import Prophet

def predict_forecast_with_prophet(df: pd.DataFrame, name:str, years_cnt=3, output_folder=""):
    """
    Vykoná predpoveď hodnôt GRP 30s pomocou modelu Prophet.

    Funkcia prijíma údaje o inzerentoch, vytvorí model časových radov a vygeneruje predpovede pre zadaný počet rokov.

    Args:
        df (pd.DataFrame): Vstupný DataFrame, ktorý obsahuje stĺpce 'Year', 'Month' a 'GRP_30s'.
            'Year' by mal byť typu int, 'Month' by mal byť typu int (1-12) a 'GRP_30s' by mal byť typu float.
        name (str): Názov inzerenta, ktorý bude zahrnutý v konečnom výsledku.
        years_cnt (int, voliteľný): Počet rokov na predpovedanie (predvolená hodnota je 3).
        output_folder (str, voliteľný): Cesta k adresáru, kde sa uloží graf s predpoveďou (predvolená hodnota je prázdny reťazec).

    Returns:
        pd.DataFrame: DataFrame s predpokladanými hodnotami GRP 30s za roky 2024, 2025, 2026 a 2027 pre daného inzerenta.
            Obsahuje stĺpce: 'Inzerent', '2024', '2025', '2026', '2027'.
    """

    df['ds'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'] + '-01')
    df['y'] = df['GRP_30s']

    prophet_data_df = df[['ds', 'y']]
    prophet_data_df = prophet_data_df.groupby('ds', as_index=False).agg({'y': 'sum'})

    # Vytvorenie a trénovanie modelu Prophet
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10,
        n_changepoints=10,
        interval_width=0.95,
        seasonality_mode='additive'
    )
    model.fit(prophet_data_df[['ds', 'y']])


    future = model.make_future_dataframe(periods=years_cnt*12 + 6, freq='M')

    # Predpovedanie
    forecast = model.predict(future)

    # Aplikujeme spätné exponenciálne transformácie a obmedzujeme záporné hodnoty
    forecast['yhat'] = forecast['yhat'].clip(lower=0)

    # Zobrazenie predpovede
    fig = model.plot(forecast)

    plt.title(f'Predpoveď GRP 30s pre {name}')
    plt.xlabel('Rok')
    plt.ylabel('GRP 30s')

    plt.ylim(bottom=0)  # Minimálna hodnota osi Y je 0

    plt.legend(['Skutočné dáta', 'Predpoveď', 'Dôveryhodná hranica'])

    if output_folder:
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        plt.savefig(os.path.join(output_folder, f"{name}.png"))

    plt.close()

    # Počítanie súčtov za roky
    forecast['year'] = forecast['ds'].dt.year
    yearly_sums = forecast.groupby('year')['yhat'].sum().reset_index()

    # Tvorba konečného DataFrame
    result_df = pd.DataFrame({
        'Advertiser': [name],
        '2024': yearly_sums[yearly_sums['year'] == 2024]['yhat'].values,
        '2025': yearly_sums[yearly_sums['year'] == 2025]['yhat'].values,
        '2026': yearly_sums[yearly_sums['year'] == 2026]['yhat'].values,
        '2027': yearly_sums[yearly_sums['year'] == 2027]['yhat'].values
    })

    return result_df


In [207]:
advertisers_list = ["KOFT S.R.O. | BROWN-FORMAN SLOVAKIA S.R.O.",
                    "GAS FAMILIA S.R.O.",
                    "ITALMARKET SLOVAKIA A.S.",
                    "MASPEX SLOVAKIA S.R.O.",
                    "MAST-JAEGERMEISTER SK S.R.O.",
                    "PERNOD RICARD SLOVAKIA S.R.O.",
                    "ST.NICOLAUS A.S.",
                    "STOCK SLOVENSKO S.R.O.",
                    "VITIS PEZINOK"]

# Popis kódu na spracovanie údajov a predpovedanie

Tento úsek kódu vykonáva spracovanie údajov o inzerentoch a realizuje predpovedanie hodnôt na základe dostupných údajov o ukazovateľoch GRP 30s. Kód je napísaný s použitím knižnice `pandas` a zahŕňa nasledujúce základné etapy:

## Import potrebných knižníc

Na začiatku kódu sú použité knižnice, ako `pandas`, na prácu s údajmi vo formáte tabuľky, a `Prophet`, ktorá sa používa na predpovedanie časových radov.

## Vytvorenie prázdneho DataFrame pre výsledky

```python
final_df = pd.DataFrame({
    'Inzerent': [],
    '2024': [],
    '2025': [],
    '2026': [],
    '2027': []
})
```

- **final_df**: Toto je prázdny DataFrame, ktorý sa bude používať na uchovávanie výsledkov predpovedania pre každého inzerenta za uvedené roky. Stĺpce predstavujú:
  - Inzerent: meno inzerenta.
  - 2024, 2025, 2026, 2027: predpokladané hodnoty GRP 30s za každý z týchto rokov.

## Iterácia cez zoznam inzerentov

```python
for advert in advertisers_list:
```
- Cyklus prechádza každým inzerentom zo zoznamu advertisers_list. V závislosti od obsahu mena inzerenta sa vyberie príslušný DataFrame.

## Kontrola prítomnosti všetkých mesiacov

```python
for year in range(start_year, end_year + 1):
    existing_months = df[df['Year'] == year]['Month'].unique()
    missing_months = set(months) - set(existing_months)
```
- Pre každý rok z rozsahu sa vykonáva kontrola na prítomnosť unikátnych mesiacov.
- Pomocou rozdielu množín sa určuje, ktoré mesiace chýbajú v údajoch.
- Pre každý chýbajúci mesiac sa vytvára nový DataFrame s nulovými hodnotami pre GRP_30s.
- Nové riadky sa pridávajú do existujúceho DataFrame pomocou pd.concat, aby sa zabezpečila prítomnosť údajov za každý mesiac v každom roku.

## Predpovedanie pomocou modelu Prophet
```python
res = predict_forecast_with_prophet(df, advert, 3, "/content/drive/MyDrive/dentsu")
```
- Volá sa funkcia predict_forecast_with_prophet, ktorá trénuje model Prophet na pripravených údajoch a vracia predpoveď na 3 roky dopredu.
- Výsledky predpovedania sa pridávajú do konečného DataFrame final_df.

In [ ]:
final_df = pd.DataFrame({
    'Advertiser': [],
    '2024': [],
    '2025': [],
    '2026': [],
    '2027': []
})

for advert in advertisers_list:
    if "|" not in advert:
        df = data.query(f"Advertiser == '{advert}'")
    else:
        advert1, advert2 = advert.split(" | ")
        df = data.query(f"Advertiser == '{advert1}' | Advertiser == '{advert2}'")

    start_year = 2020
    end_year = 2024
    months = [str(m).zfill(2) for m in range(1, 13)]

    # Prechádzanie všetkými rokmi v určenom rozsahu
    for year in range(start_year, end_year + 1):
        # Získanie jedinečných mesiacov pre daný rok
        existing_months = df[df['Year'] == year]['Month'].unique()

        # Kontrola na chýbajúce mesiace
        missing_months = set(months) - set(existing_months)

        # Vytvorenie záznamov pre chýbajúce mesiace
        for month in missing_months:
            new_row = {
                'Advertiser': [advert],
                'Brand': ["---"],
                'Year': [year],
                'Month': [month],
                'GRP_30s': [0]
            }
            df = pd.concat([df, pd.DataFrame(new_row)], ignore_index=True)

    res = predict_forecast_with_prophet(df, advert, 3, "/content/drive/MyDrive/dentsu")
    final_df = pd.concat([final_df, res], ignore_index=True)


In [216]:
final_df

,Advertiser,2024,2025,2026,2027
0,KOFT S.R.O. | BROWN-FORMAN SLOVAKIA S.R.O.,1015.166025,1217.793992,1462.996818,1711.004276
1,GAS FAMILIA S.R.O.,436.077662,911.259887,993.807167,1086.378903
2,ITALMARKET SLOVAKIA A.S.,593.700162,1384.656878,1656.796433,1928.413960
3,MASPEX SLOVAKIA S.R.O.,579.697301,965.037359,1181.290573,1395.683232
4,MAST-JAEGERMEISTER SK S.R.O.,3785.268168,4377.309753,4494.846509,4683.170946
5,PERNOD RICARD SLOVAKIA S.R.O.,4028.083177,4104.401570,4066.609445,4116.878197
6,ST.NICOLAUS A.S.,1936.869876,2140.999786,2352.351309,2562.971043
7,STOCK SLOVENSKO S.R.O.,2784.273776,3836.109747,4369.174622,4900.724061
8,VITIS PEZINOK,474.424486,1077.300249,1297.327884,1518.239407


In [ ]:
final_df.to_excel("/content/drive/MyDrive/bonus_zadanie_res.xlsx", index=False, engine='openpyxl')